In [130]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

crash_df = pd.read_csv('/content/crashdata2022-present.csv')
vehicle_df = pd.read_csv('/content/vehiclecrashdata2022-present.csv')

In [131]:
crash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11496 entries, 0 to 11495
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CrashFactId                11496 non-null  int64  
 1   Name                       11496 non-null  object 
 2   MinorInjuries              11496 non-null  int64  
 3   ModerateInjuries           11496 non-null  int64  
 4   SevereInjuries             11496 non-null  int64  
 5   FatalInjuries              11496 non-null  int64  
 6   TcrNumber                  11496 non-null  object 
 7   CityDamageFlag             11496 non-null  bool   
 8   ShortFormFlag              11496 non-null  bool   
 9   Distance                   10895 non-null  float64
 10  CrashDateTime              11496 non-null  object 
 11  PedestrianAction           11496 non-null  object 
 12  RoadwaySurface             11496 non-null  object 
 13  RoadwayCondition           11496 non-null  obj

In [132]:
vehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25694 entries, 0 to 25693
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CrashName                   25694 non-null  object 
 1   Name                        25694 non-null  object 
 2   Sex                         18192 non-null  object 
 3   Age                         25694 non-null  int64  
 4   Speed                       0 non-null      float64
 5   VehicleDamage               24945 non-null  object 
 6   PartyCategory               25694 non-null  object 
 7   Sobriety                    25694 non-null  object 
 8   VehicleDirection            25694 non-null  object 
 9   MovementPrecedingCollision  25694 non-null  object 
 10  PartyType                   25694 non-null  object 
 11  OtherAssociatedFactor       25694 non-null  object 
 12  VehicleCount                25694 non-null  int64  
 13  ViolationCode               256

In [133]:
# label encoding CollisionType
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
crash_df['CollisionTypeConverted'] = le.fit_transform(crash_df['CollisionType'])
crash_df['CollisionTypeConverted']

,CollisionTypeConverted
0,5
1,5
2,5
3,3
4,7
...,...
11491,2
11492,3
11493,6
11494,6


In [134]:
# Label encoding for the lighting feature
crash_df['LightingConverted'] = le.fit_transform(crash_df['Lighting'])
# label encoding for the weather feature
crash_df['WeatherConverted'] = le.fit_transform(crash_df['Weather'])
# label encoding for the road surface feature
crash_df['RoadwaySurfaceConverted'] = le.fit_transform(crash_df['RoadwaySurface'])

In [135]:
# convert 'MinorInjuries', 'ModerateInjuries', 'SevereInjuries', 'FatalInjuries' columns to a single column
crash_df['Severity'] = np.argmax(crash_df[['MinorInjuries', 'ModerateInjuries', 'SevereInjuries', 'FatalInjuries']], axis=1)
crash_df['Severity'].describe()

,Severity
count,11496.000000
mean,0.196590
std,0.524453
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,3.000000


In [136]:
# convert datatype of CrashDateTime column into numerical values

def map_hour_to_group(hour):
  if hour >= 0 and hour < 4:
    return 0
  elif hour >= 4 and hour < 8:
    return 1
  elif hour >= 8 and hour < 12:
    return 2
  elif hour >= 12 and hour < 16:
    return 3
  elif hour >=16  and hour < 20:
    return 4
  elif hour >= 20 and hour < 24:
    return 5

crash_df['CrashHour'] = pd.to_datetime(crash_df['CrashDateTime']).dt.hour
crash_df['CrashHourGroup'] = crash_df['CrashHour'].apply(map_hour_to_group)
crash_df['CrashHourGroup']

<ipython-input-136-7bbc2a142cb7>:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crash_df['CrashHour'] = pd.to_datetime(crash_df['CrashDateTime']).dt.hour


,CrashHourGroup
0,3
1,5
2,5
3,2
4,4
...,...
11491,5
11492,5
11493,3
11494,3


In [137]:
# combine crash_df and vehicle_df using "Name" and "CrashName"
merged_df = pd.merge(crash_df, vehicle_df, left_on='Name', right_on='CrashName', how='left')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23735 entries, 0 to 23734
Data columns (total 66 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CrashFactId                 23735 non-null  int64  
 1   Name_x                      23735 non-null  object 
 2   MinorInjuries_x             23735 non-null  int64  
 3   ModerateInjuries_x          23735 non-null  int64  
 4   SevereInjuries_x            23735 non-null  int64  
 5   FatalInjuries_x             23735 non-null  int64  
 6   TcrNumber                   23735 non-null  object 
 7   CityDamageFlag              23735 non-null  bool   
 8   ShortFormFlag               23735 non-null  bool   
 9   Distance                    22507 non-null  float64
 10  CrashDateTime               23735 non-null  object 
 11  PedestrianAction            23735 non-null  object 
 12  RoadwaySurface              23735 non-null  object 
 13  RoadwayCondition            237

In [138]:
# convert age into groups
def map_age_to_group(hour):
  if hour >= 15 and hour < 25:
    return 1
  elif hour >= 25 and hour < 30:
    return 2
  elif hour >= 30 and hour < 40:
    return 3
  elif hour >= 40 and hour < 50:
    return 4
  elif hour >=50  and hour < 60:
    return 5
  else:
    return 6

merged_df['AgeGroup'] = merged_df['Age'].apply(map_age_to_group)
merged_df['AgeGroup']

,AgeGroup
0,4
1,2
2,6
3,6
4,1
...,...
23730,4
23731,6
23732,3
23733,6


In [139]:
# label encoding for the Sobriety
merged_df['SobrietyConverted'] = le.fit_transform(merged_df['Sobriety'])
# label encoding for the Sex
merged_df['SexConverted'] = le.fit_transform(merged_df['Sex'])

In [140]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = merged_df[['Severity','CityDamageFlag', 'CrashHourGroup', 'RoadwaySurfaceConverted', 'LightingConverted', 'WeatherConverted', 'CollisionTypeConverted', 'SpeedingFlag', 'SexConverted','SobrietyConverted']], merged_df['HitAndRunFlag']


In [141]:
# Check information of X
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23735 entries, 0 to 23734
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Severity                 23735 non-null  int64
 1   CityDamageFlag           23735 non-null  bool 
 2   CrashHourGroup           23735 non-null  int64
 3   RoadwaySurfaceConverted  23735 non-null  int64
 4   LightingConverted        23735 non-null  int64
 5   WeatherConverted         23735 non-null  int64
 6   CollisionTypeConverted   23735 non-null  int64
 7   SpeedingFlag             23735 non-null  bool 
 8   SexConverted             23735 non-null  int64
 9   SobrietyConverted        23735 non-null  int64
dtypes: bool(2), int64(8)
memory usage: 1.5 MB


In [142]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [143]:
# !pip install xgboost

In [144]:
import xgboost as xgb
model = xgb.XGBClassifier()

In [145]:
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [151]:
from sklearn.metrics import accuracy_score,confusion_matrix
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
accuracy, cm

(0.8131108864172565,
 array([[4071,  283],
        [ 826,  754]]))